# Run ResNet152

In [ ]:
from __future__ import print_function 
from __future__ import division

from dataloader import RetinopathyLoader
from torch.utils import data
from ResNet import *
from run_model import *


import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models, transforms
from collections import OrderedDict
from PIL import Image
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


In [ ]:

# Batch size for training (change depending on how much memory you have)
batch_size = 4

train_set = RetinopathyLoader(root='./data/',mode='train')
train_loader = data.DataLoader(dataset=train_set,batch_size=batch_size)
test_set = RetinopathyLoader(root='./data/',mode='test')
test_loader = data.DataLoader(dataset=test_set,batch_size=batch_size)

dataloaders_dict = {'train':train_loader,
                    'val' : test_loader}



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Number of classes in the dataset
num_classes = 5

# Number of epochs to train for 
num_epochs_18 = 10
num_epochs_50 = 5


In [ ]:
# Setup the loss fxn
# Because the data imbalance, we use weight Tensor w to adjust
w = [200,40,20,7,6]
w = [1- (a/sum(w)) for a in w]
w = torch.Tensor(w).to(device)
print(w)
criterion = nn.CrossEntropyLoss(weight=w)

In [ ]:
def get_pretrained_model(model_name, num_classes,use_pretrained=True):
    model_dict = {
        'resnet18':  models.resnet18,
        'resnet34':  models.resnet34,
        'resnet50':  models.resnet50,
        'resnet101': models.resnet101,
        'resnet152': models.resnet152
    }
    
    model_ft = model_dict[model_name](pretrained=use_pretrained)          
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(num_ftrs,128)),
        ('relu', nn.ReLU()),
        ('fc2', nn.Linear(128,5))
    ]))
    input_size = 224
    
    return model_ft, input_size

In [ ]:
pretrained_152 ,_ = get_pretrained_model(model_name='resnet152',num_classes=5)
pretrained_152 = pretrained_152.to(device)

In [ ]:
pretrained_152 = torch.load('./models/vision_pretrained_152')
pretrained_152 = pretrained_152.to(device)

In [ ]:
opt_pretrained_152 = optim.SGD(pretrained_152.parameters(), lr=1e-3,weight_decay=5e-3, momentum=0.9)
lr_sch = optim.lr_scheduler.ReduceLROnPlateau(opt_pretrained_152)

pretrained_loss_list_152, pretrained_acc_train_list_152, pretrained_acc_test_list_152, best_acc = \
    run(pretrained_152, dataloaders_dict, criterion,optimizer=opt_pretrained_152\
        , scheduler=lr_sch, num_epochs=20, model_path = './models/vision_pretrained_152', best_acc = 0.7805)

In [ ]:

#torch.save(pretrained_152,'./models/vision_pretrained_152')
torch.save(pretrained_loss_list_152,'./result_list/pretrained_loss_list_152')
torch.save(pretrained_acc_train_list_152, './result_list/pretrained_acc_train_list_152')
torch.save(pretrained_acc_test_list_152, './result_list/pretrained_acc_test_list_152')